In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import geopandas as gpd
import pandas as pd
import branca
from siuba import *

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# 100 Recs - D8

## 1 - Corridor Improvements

In [2]:
corridor_gdf_d8 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="corridor"))
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                   >>head(5) #get top 5
                  )

In [3]:
corridor_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
657,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",2022-05-04,19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino
658,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",2022-05-04,15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino
659,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",2022-05-04,9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino
660,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",2022-05-04,6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino
661,269,12,intersects_shn,"LINESTRING (-117.52052 33.89185, -117.52055 33...",2022-05-04,19.0,26.0,3,12.516488,long,18.0,0.0,0.0,45.000000,2.172692,08 - San Bernardino


In [4]:
#corridor_gdf >> count(_.caltrans_district)

In [5]:
corridor_d8 = (corridor_gdf_d8
               >> select(-_.service_date)
              )

corridor_d8.explore("route_id", legend=True, tiles="CartoDB positron")

In [19]:
# for each transit operator, generate rtfiltermapper
D8_filtermapper = rt_filter_map_plot.from_gcs(itp_id=269, analysis_date=dt.date(2022,10,12))
D8_filtermapper.rt_trips >> head(5)

found parquet


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
0,269,0,2022-10-12,-3409291338062082417,1250020,18,1,shp-18-11,2022-09-11,2099-01-01,18,Mo Val College-Sunnymead,None,3,18:25:28,Westbound,16.342950,Riverside Transit Agency
1,269,0,2022-10-12,-4508282485138327398,204020,18,1,shp-18-11,2022-09-11,2099-01-01,18,Mo Val College-Sunnymead,None,3,17:35:32,Westbound,15.758981,Riverside Transit Agency
2,269,0,2022-10-12,-5817376720864208963,1682020,18,1,shp-18-11,2022-09-11,2099-01-01,18,Mo Val College-Sunnymead,None,3,10:44:56,Westbound,16.071752,Riverside Transit Agency
3,269,0,2022-10-12,3877972942018643562,1393020,18,1,shp-18-11,2022-09-11,2099-01-01,18,Mo Val College-Sunnymead,None,3,09:05:48,Westbound,16.179151,Riverside Transit Agency
4,269,0,2022-10-12,-7834366686046271089,777020,18,1,shp-18-11,2022-09-11,2099-01-01,18,Mo Val College-Sunnymead,None,3,11:35:07,Northbound,16.594916,Riverside Transit Agency


In [7]:
# get small df of route_id and route_short_name from rt_trips
D8_routenames = (D8_filtermapper.rt_trips
                 >> count(_.route_id, _.route_short_name, _.route_long_name)
                 >> select(-_.n)
                )

D8_routenames >> head(5)

,route_id,route_short_name,route_long_name
0,1,1,W. Corona-UC Riverside
1,10,10,Riverside/Watkins-Galleria
2,11,11,Moreno Valley-March ARB
3,12,12,Corona Hills Plaza/Riverside/La Cadena-Merced
4,13,13,Hunter Park Metro-Galleria


In [8]:
# inner join w gdf of slow routes
corridor_d8_names = (corridor_d8
                     >> inner_join(_,D8_routenames)
                    )

corridor_d8_names

,calitp_itp_id,route_id,category,geometry,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district,route_short_name,route_long_name
0,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino,32,Hemet Valley Mall - Mount San Jacinto College
1,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino,24,Promenade Mall - Temecula Valley Hospital
2,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino,51,UCR - Canyon Crest Towne Centre
3,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino,55,Temecula Trolley
4,269,12,intersects_shn,"LINESTRING (-117.52052 33.89185, -117.52055 33...",19.0,26.0,3,12.516488,long,18.0,0.0,0.0,45.000000,2.172692,08 - San Bernardino,12,Corona Hills Plaza/Riverside/La Cadena-Merced


In [9]:
# pass the route names as a filter parameter
routename_list_c = corridor_d8_names["route_short_name"].tolist()
routename_list_c

['32', '24', '51', '55', '12']

In [20]:
D8_filtermapper.set_filter(route_names = routename_list_c)

In [21]:
# corridor metrics for Route 12 (most linear route)
# constructed at geojson.io 
corridor_r12 = gpd.read_file('./corridors/d8_downtown_riverside.geojson')
D8_filtermapper.add_corridor(corridor_r12)
D8_filtermapper.quick_map_corridor()

In [22]:
# make maps etc
m = D8_filtermapper.segment_speed_map(corridor=True)
m

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [23]:
D8_filtermapper.corridor_metrics()

{'schedule_metric_minutes': 2.7583333333333333,
 'speed_metric_minutes': 191.51983667513542}

In [24]:
D8_filtermapper.rt_trips >> filter(_.route_id=="12")

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
350,269,0,2022-10-12,-6351256105853624293,1836020,12,0,shp-12-60,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,19:50:49,Westbound,12.307993,Riverside Transit Agency
351,269,0,2022-10-12,1601000241809086009,1665020,12,0,shp-12-60,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,10:16:34,Westbound,11.878030,Riverside Transit Agency
352,269,0,2022-10-12,-8594375813023789351,1815020,12,1,shp-12-15,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,19:19:11,Eastbound,13.476043,Riverside Transit Agency
353,269,0,2022-10-12,-7179749896761941312,2034020,12,1,shp-12-15,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,08:46:13,Eastbound,13.622559,Riverside Transit Agency
354,269,0,2022-10-12,-7032268467883880529,1715020,12,1,shp-12-15,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,12:58:12,Eastbound,12.189711,Riverside Transit Agency
355,269,0,2022-10-12,-8733444506529964265,820020,12,0,shp-12-62,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,17:47:20,Westbound,12.074455,Riverside Transit Agency
356,269,0,2022-10-12,-9015321381627235283,1567020,12,0,shp-12-60,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,11:23:34,Westbound,11.475996,Riverside Transit Agency
357,269,0,2022-10-12,-3223633136489861918,1162020,12,0,shp-12-60,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,09:17:35,Westbound,13.395880,Riverside Transit Agency
359,269,0,2022-10-12,-2924548217041873494,961020,12,1,shp-12-15,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,21:14:24,Eastbound,15.738855,Riverside Transit Agency
360,269,0,2022-10-12,2563683080415717891,887020,12,1,shp-12-16,2022-09-11,2099-01-01,12,Corona Hills Plaza/Riverside/La Cadena-Merced,None,3,22:10:38,Eastbound,14.637890,Riverside Transit Agency


In [25]:
# minutes delay per trip
191.51983667513542/32

5.984994896097982

In [14]:
# output detailed_map_view, fitered to slow segments (p20 speed below certain cutoff (8-12mph?))

## 2 - Hotspot Improvements

In [29]:
hotspot_gdf_d8 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="hotspot"))
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                   >>head(5) #get top 5
                  )

In [30]:
hotspot_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
885,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",2022-05-04,19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino
886,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",2022-05-04,15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino
887,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",2022-05-04,9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino
888,269,21,other,"LINESTRING (-117.45960 33.91186, -117.45950 33...",2022-05-04,12.0,20.0,3,11.588145,short,6.0,0.0,0.0,16.170000,2.285000,08 - San Bernardino
889,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",2022-05-04,6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino


In [31]:
hotspot_gdf_d8 = (hotspot_gdf_d8
               >> select(-_.service_date)
              )

hotspot_gdf_d8.explore("route_id", legend=True, tiles="CartoDB positron")

In [32]:
# inner join w gdf of slow routes
hotspot_d8_names = (hotspot_gdf_d8
                     >> inner_join(_,D8_routenames)
                    )

hotspot_d8_names

,calitp_itp_id,route_id,category,geometry,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district,route_short_name,route_long_name
0,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino,32,Hemet Valley Mall - Mount San Jacinto College
1,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino,24,Promenade Mall - Temecula Valley Hospital
2,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino,51,UCR - Canyon Crest Towne Centre
3,269,21,other,"LINESTRING (-117.45960 33.91186, -117.45950 33...",12.0,20.0,3,11.588145,short,6.0,0.0,0.0,16.170000,2.285000,08 - San Bernardino,21,Galleria-Pedley
4,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino,55,Temecula Trolley


In [33]:
# pass the route names as a filter parameter
routename_list_hs = hotspot_d8_names["route_short_name"].tolist()
routename_list_hs

['32', '24', '51', '21', '55']

In [34]:
D8_filtermapper.reset_filter()

In [35]:
# corridor metrics for Route 24 (most linear route)
# constructed at geojson.io 
corridor_r24 = gpd.read_file('./corridors/d8_oldtown_temecula.geojson')
D8_filtermapper.add_corridor(corridor_r24)
D8_filtermapper.quick_map_corridor()

In [36]:
D8_filtermapper.set_filter(route_names = routename_list_hs)

In [38]:
# make maps etc - mapping with the hotspot didn't return both ways of traffic
m = D8_filtermapper.segment_speed_map(corridor=False)
m

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [39]:
D8_filtermapper.corridor_metrics()

{'schedule_metric_minutes': 7.166666666666667,
 'speed_metric_minutes': 65.57034664851274}

In [40]:
D8_filtermapper.rt_trips >> filter(_.route_id=="24")

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
1014,269,0,2022-10-12,59211084171634836,228020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,17:43:04,Southbound,11.697513,Riverside Transit Agency
1015,269,0,2022-10-12,-3680707569733785287,1833020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,15:32:49,Southbound,13.836733,Riverside Transit Agency
1016,269,0,2022-10-12,2086763966394639548,1809020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,14:25:18,Southbound,10.877855,Riverside Transit Agency
1017,269,0,2022-10-12,3488047288865198596,1046020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,09:38:11,Southbound,10.949246,Riverside Transit Agency
1018,269,0,2022-10-12,8225970376924231822,1028020,24,1,shp-24-10,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,16:57:55,Northbound,11.314013,Riverside Transit Agency
1019,269,0,2022-10-12,1032029328783440736,194020,24,1,shp-24-10,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,10:58:00,Northbound,15.726436,Riverside Transit Agency
1020,269,0,2022-10-12,-4193342107904426525,990020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,16:35:19,Southbound,10.951330,Riverside Transit Agency
1021,269,0,2022-10-12,8567762161632940001,1740020,24,1,shp-24-10,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,14:40:37,Northbound,10.730103,Riverside Transit Agency
1023,269,0,2022-10-12,-3504247535873339686,1760020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,12:07:00,Southbound,11.633243,Riverside Transit Agency
1025,269,0,2022-10-12,1281839087840514620,32020,24,0,shp-24-56,2022-09-11,2099-01-01,24,Promenade Mall - Temecula Valley Hospital,None,3,13:11:44,Southbound,12.351673,Riverside Transit Agency
